In [28]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [29]:
data = pd.read_csv('FinalBalancedDataset.csv')

In [30]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 56745 entries, 0 to 56744
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  56745 non-null  int64 
 1   Toxicity    56745 non-null  int64 
 2   tweet       56745 non-null  object
dtypes: int64(2), object(1)
memory usage: 1.3+ MB


In [31]:
data.head(5)

,Unnamed: 0,Toxicity,tweet
0,0,0,@user when a father is dysfunctional and is s...
1,1,0,@user @user thanks for #lyft credit i can't us...
2,2,0,bihday your majesty
3,3,0,#model i love u take with u all the time in ...
4,4,0,factsguide: society now #motivation


In [32]:
data = data.drop("Unnamed: 0", axis = 1)

In [33]:
data.head(5)

,Toxicity,tweet
0,0,@user when a father is dysfunctional and is s...
1,0,@user @user thanks for #lyft credit i can't us...
2,0,bihday your majesty
3,0,#model i love u take with u all the time in ...
4,0,factsguide: society now #motivation


In [34]:
data['Toxicity'].value_counts()

,count
Toxicity,
0,32592
1,24153


In [40]:
import nltk
nltk.download('punkt')
nltk.download('omw-1.4')
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')
nltk.download('punkt_tab')
nltk.download('averaged_perceptron_tagger_eng')
from nltk import WordNetLemmatizer
from nltk import pos_tag, word_tokenize
from nltk.corpus import stopwords as nltk_stopwords
from nltk.corpus import wordnet

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger_eng.zip.


## Lemmatizer
1. Leaves
2. Leafs
Leaf

In [41]:
wordnet_lemmatizer = WordNetLemmatizer()

In [42]:
import re

In [44]:
def prepare_text(text):
  def get_wordnet_pos(treebank_tag):
    if treebank_tag.startswith('J'):
      return wordnet.ADJ
    elif treebank_tag.startswith('V'):
      return wordnet.VERB
    elif treebank_tag.startswith('N'):
      return wordnet.NOUN
    elif treebank_tag.startswith('R'):
      return wordnet.ADV
    else:
      return wordnet.NOUN
  text = re.sub(r'[^a-zA-Z\']', ' ', text)
  text = text.split()
  text = ' '.join(text)
  text = word_tokenize(text)
  text = pos_tag(text)
  lemma = []
  for i in text: lemma.append(wordnet_lemmatizer.lemmatize(i[0], pos = get_wordnet_pos(i[1])))
  lemma = ' '.join(lemma)
  return lemma

In [45]:
data['clean_tweets'] = data['tweet'].apply(lambda x: prepare_text(x))

In [46]:
data.head(5)

,Toxicity,tweet,clean_tweets
0,0,@user when a father is dysfunctional and is s...,user when a father be dysfunctional and be so ...
1,0,@user @user thanks for #lyft credit i can't us...,user user thanks for lyft credit i ca n't use ...
2,0,bihday your majesty,bihday your majesty
3,0,#model i love u take with u all the time in ...,model i love u take with u all the time in ur
4,0,factsguide: society now #motivation,factsguide society now motivation


Tfidf for features

In [47]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve

In [48]:
corpus = data['clean_tweets'].values.astype('U')

In [51]:
stopwords = list(nltk_stopwords.words('english'))

In [52]:
count_tf_idf = TfidfVectorizer(stop_words = stopwords)
tf_idf = count_tf_idf.fit_transform(corpus)

In [53]:
import pickle

In [54]:
pickle.dump(count_tf_idf, open("tf_idf.pkt", "wb"))

In [55]:
tf_idf_train, tf_idf_test, target_train, target_test = train_test_split(
    tf_idf, data['Toxicity'], test_size = 0.8, random_state= 42, shuffle=True
)

Create a Binary Classification Model

In [56]:
model_bayes = MultinomialNB()

In [57]:
model_bayes = model_bayes.fit(tf_idf_train, target_train)

In [58]:
y_pred_proba = model_bayes.predict_proba(tf_idf_test)[::, 1]

In [59]:
y_pred_proba

array([0.90151866, 0.27913231, 0.79020483, ..., 0.09474976, 0.20552427,
       0.32089036])

In [60]:
fpr, tpr, _ = roc_curve(target_test, y_pred_proba)

In [61]:
final_roc_auc = roc_auc_score(target_test, y_pred_proba)

In [62]:
final_roc_auc

0.9659014779038171

In [63]:
test_text = "I hate you moron"
test_tfidf = count_tf_idf.transform([test_text])
display(model_bayes.predict_proba(test_tfidf))
display(model_bayes.predict(test_tfidf))

array([[0.39921021, 0.60078979]])

array([1])

Save the model

In [64]:
pickle.dump(model_bayes, open("toxicity_model.pkt", "wb"))